# Using Open Source Vis.JS Library to PGX

## First Step: Get a session in PGX server.

In [1]:
import json
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

def generateToken(base_url, username, password):
    from urllib.request import Request, urlopen
    from urllib.error import HTTPError

    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)

    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')

    except HTTPError as err:

        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))

        os._exit(1)

baseUrl = 'https://localhost:7007'
username = 'workshop022'
password = 'welcome1'

token=generateToken(baseUrl, username, password)

In [2]:
print(token)

eyJraWQiOiJEYXRhYmFzZVJlYWxtIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJXT1JLU0hPUDAyMiIsInJvbGVzIjpbIkdSQVBIX0RFVkVMT1BFUiIsIkdSQVBIX0FETUlOSVNUUkFUT1IiXSwiZ2VuZXJhbF9wZXJtaXNzaW9ucyI6WyJQR1hfU0VTU0lPTl9HRVRfUFVCTElTSEVEX0dSQVBIIiwiUEdYX1NFUlZFUl9HRVRfSU5GTyIsIlBHWF9TRVNTSU9OX01PRElGWV9NT0RFTCIsIlBHWF9TRVNTSU9OX1JFQURfTU9ERUwiLCJQR1hfU0VTU0lPTl9DUkVBVEUiLCJQR1hfU0VSVkVSX01BTkFHRSIsIlBHWF9TRVNTSU9OX05FV19HUkFQSCJdLCJpc3MiOiJvcmFjbGUucGcuaWRlbnRpdHkucmVzdC5BdXRoZW50aWNhdGlvblNlcnZpY2UiLCJleHAiOjE2MTYzOTk3NzgsIm51bV90b2tlbl9yZWZyZXNoIjowLCJmaWxlX3Blcm1pc3Npb25zIjp7IlwvaG9tZVwvb3JhY2xlXC9Eb3dubG9hZHNcL2JzYW4iOlsiUkVBRCJdfX0.pM-Ne-_0D2_xEf_wpOMCNFpZ96igonTCeJwBGM3VCdPuvSxgjXR5wtTBUaEcLEX5ZqYLoe4zWHNiVOYnFlrAKH298AkQKfm1Lcclejc5ANvsSS6rvxDmbnr9KG7ZGsMsC3pSopv6p9AhjUbOrJmmosIkVUD7XSiWQiwO04xn0_qJ4IyCib2Mqb_n-uYKJfSerd67nXBF2JFoA1a41Ql-kvrlwjzAhWpfgzWAH3EBQyFz-sxNt5MbBid8pLxfX3RNQR-d_GOiUwhKudiU731isR9ILRasdRUd7JkQNk9TooA5vuDomn40IIGYAF9w1QviUbq_fJCVcxPP7fug6_D6mw


In [3]:
try:
    import pypgx as pgx

    session = pgx.get_session(base_url=baseUrl, token=token)
    instance = session.server_instance
    analyst = session.create_analyst()

except SystemExit:
    os._exit(0)
    

In [4]:
print(session)

PgxSession(id: 1a5cadf9-2b36-4729-a6d2-4e10f412e6b1, name: pypgx)


## Load JSON File with Property Graph Definition

In [6]:
# read Graph
graph = session.read_graph_with_properties("./bank.json")


RuntimeError: JVM exception occurred: java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC java.util.concurrent.ExecutionException
Java stack trace:
    java.util.concurrent.ExecutionException: java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC
    java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:357)
    java.util.concurrent.CompletableFuture.get(CompletableFuture.java:1908)
    oracle.pgx.api.PgxFuture.get(PgxFuture.java:99)
    oracle.pgx.api.PgxSession.readGraphWithProperties(PgxSession.java:1959)
Caused by:
    java.lang.Exception: ORA-00942: table or view does not exist
 code: PGX-ERROR-2R8OO37VPPJTC
    oracle.pgx.common.marshalers.ExceptionMarshaler.toUnserializedException(ExceptionMarshaler.java:110)
    oracle.pgx.common.marshalers.ExceptionMarshaler.unmarshal(ExceptionMarshaler.java:123)
    oracle.pgx.client.RemoteUtils.parseExceptionalResponse(RemoteUtils.java:130)
    or

In [ ]:
print('Edges number: ' + str(graph.num_edges) + ' Nodes number: '+ str(graph.num_vertices))

In [ ]:
# get result
# node data
pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRST_NAME, x.CITY
  MATCH (x)-[]-()
""")

#['size', 'value', 'title', 'x', 'y', 'label', 'color']

node_id = []
node_title =[]
node_value = []
node_label = []

for i in pgxResultSetNode:
    size = i[0]
    if size not in node_id:
        node_id.append(size)
        node_label.append(i[1])
        if i[2] != '':
           node_title.append(i[2])
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' FIRST_NAME: ' + i[2])
        elif i[3] != '':
           node_title.append(i[3])
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' CITY: ' + i[3])
        else:
           node_title.append('n/a') 
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' title: n/a')
        
        
        
# edge data

pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")


edge_list = []
for i in pgxResultSetEdge:
    edge_tuple = (i[0], i[1], i[2])
    edge_list.append(edge_tuple)
    print('Edge: src: ' + str(i[0]) + ' dst: ' + str(i[1]) + ' label: ' + i[2])

## Open Source Vis.JS Visualization using Network Python Library

In [ ]:
from pyvis.network import Network

g = Network(notebook=True, height = '800px', width = '100%' , directed = True)
g.options = {
          "nodes": {
            "scaling": {
              "min": 16,
              "max": 32,
            },
          },
          "edges": {
            "color": "GRAY",
            "smooth": "false",
          },
          "physics": {
            "barnesHut": { "gravitationalConstant": -30000 },
            "stabilization": { "iterations": 2500 },
          }
}

g.add_nodes(node_id,label=node_label,title=node_title)
g.add_edges(edge_list)
#g.show_buttons()
g.show('graph.html')

## Another Vis.JS Visualization using Network Python Library

In [ ]:
from pyvis.network import Network

g2 = Network(notebook=True, height = '800px', width = '100%' , directed = True)
g2.options = {
          "nodes": {
            "scaling": {
              "min": 16,
              "max": 32,
            },
          },
          "edges": {
            "color": "GRAY",
            "smooth": "false",
          },
          "physics": {
            "barnesHut": { "gravitationalConstant": -30000 },
            "stabilization": { "iterations": 2500 },
          },
          "groups": {
            "CUST": {
              "shape": "triangle",
              "color": "#FF9900",
            },
            "PHONENUMBERS": {
              "shape": "dot",
              "color": "#2B7CE9",
            },
            "ADDRESS": {
              "shape": "dot",
              "color": "#5A1E5C",
            },
            "ACCOUNTS": {
              "shape": "square",
              "color": "#C5000B",
            },
            "SSN": {
              "shape": "square",
              "color": "#109618",
            },
          },
}

# node data
pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRSTNAME, x.CITY
  MATCH (x)-[]-()
""")

for i in pgxResultSetNode:
    if i[2] != '':
       g2.add_node(i[0],label=i[1],group=i[1],title=i[2])
       #print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' FIRST_NAME: ' + i[2])
    elif i[3] != '':
       g2.add_node(i[0],label=i[1],group=i[1],title=i[3])
       #print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' CITY: ' + i[3])
    else:
       g2.add_node(i[0],label=i[1],group=i[1],title='n/a')
       #print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' title: n/a')
        
# edge data

pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")

for i in pgxResultSetEdge:
    g2.add_edge(i[0],i[1],title=i[2])
    #print('Edge: src: ' + str(i[0]) + ' dst: ' + str(i[1]) + ' label: ' + i[2])

#g.show_buttons()
g2.show('graph.html')

## Another Vis.JS Visualization using NetworkX Python Library

In [ ]:
from pyvis.network import Network
import networkx as nx

nx_graph = nx.Graph()

pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRSTNAME, x.CITY
  MATCH (x)-[]-()
""")

for i in pgxResultSetNode:
    if i[2]:
      nx_graph.add_node(i[0], size=20, title=i[2], group=i[1])
    else:
      nx_graph.add_node(i[0], size=20, title=i[3], group=i[1])
    
# edge data
pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")

edge_list = []
for i in pgxResultSetEdge:
    nx_graph.add_edge(i[0], i[1], weight=5)
    
nt = Network(notebook=True, height = '800px', width = '100%')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.show('nx.html')

## Destroy Graph In PGX Memory 

In [ ]:
session.destroy()